In [15]:
import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import sqlite3

In [16]:
# Connect to sqlite
conn = sqlite3.connect('energy.db')

# Load weather dataset
df_w = pd.read_sql_query("SELECT * FROM Vilnius_weather", conn)
df_w['Date'] = pd.to_datetime(df_w['obs_time_utc'].str.split(' ').str[0], format='%Y-%m-%d')
df_w['Hour'] = pd.to_datetime(df_w['obs_time_utc']).dt.hour

# Load generation dataset
df_g = pd.read_sql_query("SELECT * FROM Vilnius_generation", conn)
df_g = pd.melt(df_g, id_vars=['Parameter', 'Day'], var_name='Hour', value_name='Average P- per Hour')
df_g['Date'] = pd.to_datetime(df_g['Day'], format='%d/%m/%Y')

# Load consumption dataset
df_c = pd.read_sql_query("SELECT * FROM Vilnius_consumption", conn)
df_c = pd.melt(df_c, id_vars=['Parameter', 'Day'], var_name='Hour', value_name='Average P+ per Hour')
df_c['Date'] = pd.to_datetime(df_c['Day'], format='%d/%m/%Y')

# Close sqlite connection
conn.close()

In [17]:
# Group generation and consumption by hour, calculate the average
generation_hourly = df_g.groupby('Hour')['Average P- per Hour'].mean()
consumption_hourly = df_c.groupby('Hour')['Average P+ per Hour'].mean()

# Create traces for generation and consumption
trace_generation = go.Scatter(
    x=generation_hourly.index,
    y=generation_hourly.values,
    mode='lines',
    name='Generation (P-)',
    line=dict(color='green')
)

trace_consumption = go.Scatter(
    x=consumption_hourly.index,
    y=consumption_hourly.values,
    mode='lines',
    name='Consumption (P+)',
    line=dict(color='red')
)

# Combine traces into data list
traces = [trace_generation, trace_consumption]

layout = go.Layout(
    title='Hourly Generation and Consumption Trends',
    xaxis=dict(tickfont=dict(size=10)),
    yaxis=dict(title='Average Energy (kWh)'),
    hovermode='closest',
    legend=dict(orientation="h")
)

# Reference data source
source_annotation = go.layout.Annotation(
    text='Source: Ignitis 2023',
    xref='paper', yref='paper',
    x=0, y=-0.25,
    showarrow=False,
    font=dict(size=10, color='grey')
)

layout['annotations'] = [source_annotation]

fig1 = go.Figure(data=traces, layout=layout)
fig1.update_layout(template="plotly_dark")

fig1.show()


In [18]:
# Group generation and consumption data by day, calculate the sum
generation_daily = df_g.groupby('Date')['Average P- per Hour'].sum()
consumption_daily = df_c.groupby('Date')['Average P+ per Hour'].sum()

# Create traces for generation and consumption
trace_generation = go.Scatter(
    x=generation_daily.index,
    y=generation_daily.values,
    mode='lines',
    name='Generation (P-)',
    line=dict(color='green')
)

trace_consumption = go.Scatter(
    x=consumption_daily.index,
    y=consumption_daily.values,
    mode='lines',
    name='Consumption (P+)',
    line=dict(color='red')
)

# Combine traces into data list
traces = [trace_generation, trace_consumption]

layout = go.Layout(
    title='Daily Generation and Consumption Comparison',
    xaxis=dict(),
    yaxis=dict(title='Total Energy (kWh)'),
    hovermode='closest',
    legend=dict(orientation="h")
)

# Reference data source
source_annotation = go.layout.Annotation(
    text='Source: Ignitis 2023',
    xref='paper', yref='paper',
    x=0, y=-0.25,
    showarrow=False,
    font=dict(size=10, color='grey')
)

layout['annotations'] = [source_annotation]

fig2 = go.Figure(data=traces, layout=layout)
fig2.update_layout(template="plotly_dark")

fig2.show()

In [19]:
# Calculate net energy balance for each hour
df_g['Net Energy Balance'] = df_g['Average P- per Hour'] - df_c['Average P+ per Hour']

# Group net energy balance data by day, calculate the sum
net_energy_balance = df_g.groupby('Date')['Net Energy Balance'].sum()

# Create trace for net energy balance
trace_net_balance = go.Scatter(
    x=net_energy_balance.index,
    y=net_energy_balance.values,
    mode='lines',
    name='Net Energy Balance',
    line=dict(color='purple')
)

layout = go.Layout(
    title='Net Energy Balance Over Time',
    xaxis=dict(title='Date'),
    yaxis=dict(title='Net Energy Balance (kWh)'),
    hovermode='closest',
    legend=dict(orientation="h")
)

# Reference data source
source_annotation = go.layout.Annotation(
    text='Source: Ignitis 2023',
    xref='paper', yref='paper',
    x=0, y=-0.25,
    showarrow=False,
    font=dict(size=10, color='grey')
)

layout['annotations'] = [source_annotation]
fig3 = go.Figure(data=[trace_net_balance], layout=layout)
fig3.update_layout(template="plotly_dark")

fig3.show()

In [20]:
# Weather data daily averages
weather_average = df_w.groupby(df_w['Date'])[['air_temperature', 'wind_speed', 'cloud_cover']].mean()

# Figure with subplots
fig4 = make_subplots(rows=1, cols=3, subplot_titles=("Air Temperature vs. Energy", "Wind Speed vs. Energy", "Cloud cover vs. Energy"))

# Plot air temperature vs. energy generation/consumption
fig4.add_trace(go.Scatter(x=weather_average['air_temperature'], y=generation_daily, mode='markers', name='Generation', marker=dict(color='green', opacity=0.5)), row=1, col=1)
fig4.add_trace(go.Scatter(x=weather_average['air_temperature'], y=consumption_daily, mode='markers', name='Consumption', marker=dict(color='red', opacity=0.5)), row=1, col=1)
fig4.update_xaxes(title_text="Air Temperature (°C)", row=1, col=1)
fig4.update_yaxes(title_text="Energy (kWh)", row=1, col=1)

# Plot wind speed vs. energy generation/consumption
fig4.add_trace(go.Scatter(x=weather_average['wind_speed'], y=generation_daily, mode='markers', name='Generation', marker=dict(color='green', opacity=0.5)), row=1, col=2)
fig4.add_trace(go.Scatter(x=weather_average['wind_speed'], y=consumption_daily, mode='markers', name='Consumption', marker=dict(color='red', opacity=0.5)), row=1, col=2)
fig4.update_xaxes(title_text="Wind Speed (m/s)", row=1, col=2)
fig4.update_yaxes(title_text="Energy (kWh)", row=1, col=2)

# Plot cloud cover vs. energy generation/consumption
fig4.add_trace(go.Scatter(x=weather_average['cloud_cover'], y=generation_daily, mode='markers', name='Generation', marker=dict(color='green', opacity=0.5)), row=1, col=3)
fig4.add_trace(go.Scatter(x=weather_average['cloud_cover'], y=consumption_daily, mode='markers', name='Consumption', marker=dict(color='red', opacity=0.5)), row=1, col=3)
fig4.update_xaxes(title_text="Cloud cover (%)", row=1, col=3)
fig4.update_yaxes(title_text="Energy (kWh)", row=1, col=3)

# Reference data source
annotation = go.layout.Annotation(
    text='Source: Ignitis 2023',
    xref='paper', yref='paper',
    x=0, y=-0.25,
    showarrow=False,
    font=dict(size=10, color='grey')
)
fig4.layout.annotations = [annotation]

fig4.update_layout(title="Weather vs. Energy", showlegend=False, template='plotly_dark')
fig4.show()


In [21]:
import plotly.express as px
from jinja2 import Template
import json

figs = [fig1, fig2, fig3, fig4]  

# Convert each Plotly figure to JSON
fig_jsons = [fig.to_json() for fig in figs]

# Load the Jinja2 template
input_template_path = "html_samples/input.html"
with open(input_template_path) as template_file:
    j2_template = Template(template_file.read())
rendered_html = j2_template.render(fig_jsons=fig_jsons)

# Save HTML
output_html_path = "html_samples/Vilnius.html"
with open(output_html_path, "w", encoding="utf-8") as output_file:
    output_file.write(rendered_html)